In [1]:
import os
import sys
sys.path.insert(0, "../")

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import random
from tqdm import tqdm_notebook as tqdm

from learning.treelstm.utils import load_word_vectors
from learning.treelstm.trainer import Trainer
from learning.treelstm.metrics import Metrics
from learning.treelstm.model import *
from learning.treelstm.vocab import Vocab
import learning.treelstm.Constants as Constants
from learning.treelstm.dataset import QGDataset
from learning.treelstm.scripts.preprocess_lcquad import build_vocab
from itertools import product

data_path = '../learning/treelstm/data/lc_quad/'
train_path = data_path + 'train/'
dev_path = data_path + 'dev/'
test_path = data_path + 'test/'
checkpoints_path = '../learning/treelstm/checkpoints'

In [2]:
class Struct: pass
args = Struct()
args.seed = 41
args.cuda = False
args.batchsize = 20
args.mem_dim = 150
args.hidden_dim = 50
args.num_classes = 2
args.input_dim = 300
args.sparse = False
args.lr = 0.01
args.wd = 1e-4

args.epochs = 15

In [3]:
torch.manual_seed(args.seed)
random.seed(args.seed)

In [4]:
torch.get_num_threads()
torch.set_num_threads(2)

### vocab

In [5]:
# mapping words to indexes
vocab = Vocab(
    os.path.join(data_path, 'dataset.vocab'),
    [Constants.PAD_WORD, Constants.UNK_WORD, Constants.BOS_WORD, Constants.EOS_WORD]
)

In [20]:
len(vocab.idxToLabel)

8057

In [17]:
try:
    emb = torch.load('glove_lc_merged_emb.pth')
except:
    emb = torch.Tensor(vocab.size(), 300).normal_(-0.05, 0.05)
    # zero out the embeddings for padding and other special words if they are absent in vocab
    for idx, item in enumerate([Constants.PAD_WORD, Constants.UNK_WORD, Constants.BOS_WORD, Constants.EOS_WORD]):
        emb[idx].zero_()

    with open('../learning/treelstm/data/glove.840B.300d.txt', 'r') as file:
        for line in tqdm(file):
            contents = line.rstrip('\n').split(' ')
            idx = vocab.getIndex(contents[0])
            if(idx):
                emb[idx] = torch.Tensor(list(map(float, contents[1:])))

    torch.save(emb, 'glove_lc_merged_emb.pth')

### sets

In [6]:
train_set = QGDataset(train_path, vocab, args.num_classes)
dev_set = QGDataset(dev_path, vocab, args.num_classes)
test_set = QGDataset(test_path, vocab, args.num_classes)

100%|██████████| 1090/1090 [00:00<00:00, 48279.12it/s]


In [7]:
7896+2265+1090

11251

### model

In [8]:
similarity = DASimilarity(args.mem_dim, args.hidden_dim, args.num_classes)
#similarity = CosSimilarity(1)

In [9]:
# left - a - sent
# right - b - query

class SimilarityEncoders(nn.Module):
    def __init__(self, vocab_size, in_dim, mem_dim, similarity, sparsity):
        super(SimilarityEncoders, self).__init__()
        self.emb = nn.Embedding(vocab_size, in_dim, padding_idx=Constants.PAD, sparse=sparsity)
        self.sent_treelstm = ChildSumTreeLSTM(in_dim, mem_dim)
        self.query_treelstm = ChildSumTreeLSTM(in_dim, mem_dim)
        self.similarity = similarity

    def forward(self, ltree, linputs, rtree, rinputs):
        linputs = self.emb(linputs)
        rinputs = self.emb(rinputs)
        lstate, lhidden = self.sent_treelstm(ltree, linputs)
        rstate, rhidden = self.query_treelstm(rtree, rinputs)
        output = self.similarity(lstate, rstate)
        return output

In [10]:
model = SimilarityEncoders(vocab.size(), args.input_dim, args.mem_dim, similarity, args.sparse)

In [11]:
criterion = nn.KLDivLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=args.lr, weight_decay=args.wd)

In [12]:
# model.emb.weight.data.copy_(emb)

NameError: name 'emb' is not defined

In [13]:
trainer = Trainer(args, model, criterion, optimizer)
metrics = Metrics(args.num_classes)

In [14]:
model.parameters()

<generator object Module.parameters at 0x7fa3a4580f68>

### Treino

In [20]:
# train_loss, train_pred = trainer.test(train_set)
# print('train_loss:', train_loss)
# print('train_pred:', train_pred)

In [21]:
for epoch in range(args.epochs):
    train_loss = trainer.train(train_set)
    train_loss, train_pred = trainer.test(train_set)
    print('train_loss:', train_loss)
    print('train_pred:', train_pred)
    checkpoint = {'model': trainer.model.state_dict(), 'optim': trainer.optimizer,
                  'args': args, 'epoch': epoch}
    torch.save(checkpoint, 'checkpoint_' + str(epoch) + '.pth')

Training epoch 1:   0%|          | 0/7896 [00:00<?, ?it/s]/home/mateus/TCC/envSQG/lib/python3.5/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/mateus/TCC/envSQG/lib/python3.5/site-packages/torch/nn/functional.py:1339: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
../learning/treelstm/model.py:66: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.log_softmax(self.wp(out))
/home/mateus/TCC/envSQG/lib/python3.5/site-packages/torch/nn/functional.py:1932: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will

train_loss: tensor(0.1514)
train_pred: tensor([1.9249, 1.1357, 1.9325,  ..., 1.9290, 1.1791, 1.9251])


Training epoch 3:   0%|          | 3/7896 [00:00<05:46, 22.78it/s]

train_loss: tensor(0.1163)
train_pred: tensor([1.9398, 1.0694, 1.9587,  ..., 1.9484, 1.0891, 1.9549])


Training epoch 4:   0%|          | 2/7896 [00:00<07:18, 18.02it/s]

...

train_loss: tensor(0.0268)
train_pred: tensor([1.9955, 1.0018, 1.9967,  ..., 1.9970, 1.0015, 1.9973])


Training epoch 15:   0%|          | 3/7896 [00:00<05:12, 25.24it/s]

train_loss: tensor(0.0287)
train_pred: tensor([1.9968, 1.0020, 1.9971,  ..., 1.9975, 1.0014, 1.9977])


Testing epoch  15: 100%|██████████| 7896/7896 [01:18<00:00, 100.45it/s]

train_loss: tensor(0.0259)
train_pred: tensor([1.9966, 1.0016, 1.9972,  ..., 1.9978, 1.0012, 1.9979])


### Teste

In [23]:
for epoch in range(args.epochs):
    checkpoint = torch.load('checkpoint_' + str(epoch) + '.pth')
    model.load_state_dict(checkpoint['model'])
    loss, pred = trainer.test(dev_set)
    print('epoch', epoch, metrics.f1(pred.numpy(), dev_set.labels))

Testing epoch  0:   0%|          | 0/2265 [00:00<?, ?it/s]../learning/treelstm/trainer.py:50: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  linput, rinput = Var(lsent, volatile=True), Var(rsent, volatile=True)
../learning/treelstm/trainer.py:51: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  target = Var(map_label_to_target(label, dataset.num_classes), volatile=True)
/home/mateus/TCC/envSQG/lib/python3.5/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/mateus/TCC/envSQG/lib/python3.5/site-packages/torch/nn/functional.py:1339: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
../learning/treelstm/model.py:66: UserWarning: Implicit dimen

epoch 0 (0.8488633220603211, 0.848108381892653, 0.8484479832602387)


Testing epoch  0:   0%|          | 8/2265 [00:00<00:28, 79.61it/s]

epoch 1 (0.8441505672148859, 0.8456562407873196, 0.8445672589748006)


Testing epoch  0:   0%|          | 8/2265 [00:00<00:29, 77.72it/s]

...

epoch 12 (0.8193450753729594, 0.8205679558669485, 0.817169754814719)


Testing epoch  0:   0%|          | 9/2265 [00:00<00:26, 86.55it/s]

epoch 13 (0.7883769119029309, 0.7857785224585547, 0.7795747691720813)


Testing epoch  0: 100%|██████████| 2265/2265 [00:26<00:00, 85.88it/s] 

epoch 14 (0.8013806161492643, 0.7995218814880791, 0.7937609346158474)


In [15]:
checkpoint = torch.load('checkpoint_14.pth')
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [16]:
model.emb.weight.data

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 3.1492e-06, -7.0746e-40,  6.9912e-40,  ..., -3.4392e-03,
         -4.6204e-04, -2.1826e-03],
        ...,
        [-7.7178e-02, -5.1056e-02, -8.5519e-02,  ..., -3.8988e-02,
         -4.5611e-03,  6.5343e-03],
        [-2.9133e-04,  3.0484e-04,  6.9033e-04,  ...,  3.2669e-04,
          4.2872e-05,  5.6230e-05],
        [ 8.6759e-02,  1.3279e-02, -2.2038e-01,  ...,  1.2827e-02,
         -1.1806e-01, -1.4734e-01]])

In [22]:
metrics.f1(pred.numpy(), dev_set.labels)

(0.8013806161492643, 0.7995218814880791, 0.7937609346158474)